In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, time
import pytz

# Mock data generation function (replace with real API calls)
def fetch_play_store_data():
    # In a real implementation, this would connect to Google Play Store API
    # For demo purposes, we'll create mock data
    categories = ['GAME', 'SOCIAL', 'COMMUNICATION']
    app_types = ['Free', 'Paid']
    
    data = []
    for category in categories:
        for app_type in app_types:
            for _ in range(50):  # 50 apps per category per type
                installs = np.random.randint(10000, 10000000)
                revenue = np.random.uniform(10000, 1000000)
                android_ver = round(np.random.uniform(4.1, 10.0), 1)
                size = np.random.randint(15, 100)
                name = f"App{np.random.randint(1, 1000)}"[:30]
                
                data.append({
                    'Category': category,
                    'Type': app_type,
                    'Installs': installs,
                    'Revenue': revenue,
                    'AndroidVersion': android_ver,
                    'Size': size,
                    'ContentRating': 'Everyone',
                    'AppName': name
                })
    
    return pd.DataFrame(data)

# Apply all required filters
def filter_data(df):
    # Current time check (only between 1 PM to 2 PM IST)
    ist = pytz.timezone('Asia/Kolkata')
    current_time = datetime.now(ist).time()
    
    if not (time(13, 0) <= current_time <= time(14, 0)):
        return None
    
    # Apply all filters
    filtered = df[
        (df['Installs'] >= 10000) &
        (df['Revenue'] >= 10000) &
        (df['AndroidVersion'] > 4.0) &
        (df['Size'] > 15) &
        (df['ContentRating'] == 'Everyone') &
        (df['AppName'].str.len() <= 30)
    ]
    
    return filtered

# Process data and create visualization
def create_dual_axis_chart():
    # Fetch and filter data
    df = fetch_play_store_data()
    filtered_df = filter_data(df)
    
    if filtered_df is None:
        print("Visualization only available between 1 PM to 2 PM IST")
        return
    
    # Get top 3 categories by install count
    top_categories = filtered_df.groupby('Category')['Installs'].sum().nlargest(3).index
    
    # Filter for top categories
    top_cat_df = filtered_df[filtered_df['Category'].isin(top_categories)]
    
    # Group by category and app type
    grouped = top_cat_df.groupby(['Category', 'Type']).agg({
        'Installs': 'mean',
        'Revenue': 'mean'
    }).reset_index()
    
    # Pivot for plotting
    pivot_installs = grouped.pivot(index='Category', columns='Type', values='Installs')
    pivot_revenue = grouped.pivot(index='Category', columns='Type', values='Revenue')
    
    # Create figure and axis
    fig, ax1 = plt.subplots(figsize=(12, 8))
    
    # Plot installs on primary axis
    colors = ['#1f77b4', '#ff7f0e']
    pivot_installs.plot(kind='bar', ax=ax1, color=colors, alpha=0.7)
    ax1.set_ylabel('Average Installs', color='#1f77b4')
    ax1.tick_params(axis='y', labelcolor='#1f77b4')
    ax1.set_xlabel('App Category')
    ax1.set_title('Comparison of Average Installs and Revenue: Free vs Paid Apps (Top 3 Categories)')
    
    # Create secondary axis for revenue
    ax2 = ax1.twinx()
    for i, (col, color) in enumerate(zip(pivot_revenue.columns, colors)):
        ax2.plot(
            [x + i * 0.2 - 0.1 for x in range(len(pivot_revenue))],
            pivot_revenue[col],
            'o--',
            color=color,
            markersize=8,
            label=f'{col} Revenue'
        )
    ax2.set_ylabel('Average Revenue ($)', color='#2ca02c')
    ax2.tick_params(axis='y', labelcolor='#2ca02c')
    
    # Add legend and adjust layout
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Run the visualization
create_dual_axis_chart()

Visualization only available between 1 PM to 2 PM IST
